In [10]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

# Django ORM CRUD Operations Tutorial

CRUD stands for Create, Read, Uptimestamp, and Delete - the four basic operations for database management systems. In this tutorial, we'll explore how to perform these operations using Django ORM with the NewsData model.

## NewsData Model Structure

Let's first examine the structure of our NewsData model to understand its fields.

In [11]:
# Inspect the NewsData model fields
print('NewsData Model Fields:')
for field in NewsData._meta.get_fields():
    print(f'- {field.name}: {field.get_internal_type()}')

# Display total count of news entries
print(f'\nTotal news entries in database: {NewsData.objects.count()}')

NewsData Model Fields:
- id: CharField
- timestamp: DateField
- category: CharField
- title: TextField
- content: TextField
- sentiment: FloatField
- top_key_freq: TextField
- tokens: TextField
- tokens_v2: TextField
- entities: TextField
- token_pos: TextField
- link: CharField
- photo: CharField

Total news entries in database: 13


## 1. Create (C) - Adding New Data

The 'Create' operation allows us to insert new records into the database. Here are different ways to create new NewsData objects.

In [14]:
# Method 1: Create using the constructor and save() method
new_news1 = NewsData(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news1",
    category="測試類別",
    content="This is a sample content about technology news for demonstration.",
    timestamp=datetime.now().date(),
    photo="https://example.com/image1.jpg"
)
# Uncomment to save: new_news1.save()

# Method 2: Using objects.create() method
# This creates and saves in one step
new_news2 = NewsData.objects.create(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news2",
    category="測試類別",
    content="This is sample content about international news for demonstration.",
    timestamp=datetime.now().date(),
    photo=None  # Shows that photo can be optional
)

# Method 3: Bulk creation (for adding multiple records at once)
news_list = [
    NewsData(
        title=f"Bulk Example {i}", 
        link=f"https://example.com/bulk{i}",
        category="測試類別",
        content=f"Bulk created content {i}",
        timestamp=datetime.now().date()
    ) for i in range(3, 6)
]
# Uncomment to save: NewsData.objects.bulk_create(news_list)

print(f"Created news item: {new_news2.title} (ID: {new_news2.id})")
print("Note: The other examples are commented out to prevent adding test data to your database.")

Created news item: 馬斯克蒞臨高科大 (ID: )
Note: The other examples are commented out to prevent adding test data to your database.


In [15]:

# Clean up - delete the example we created
new_news2.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 2. Read (R) - Querying Data

The 'Read' operation retrieves data from the database. Django ORM provides powerful ways to query data.

In [ ]:
# Basic querying examples

# 1. Get all records
all_news = NewsData.objects.all()
print(f"Total news items: {all_news.count()}")


In [ ]:

# 2. Get a specific record by ID
try:
    first_id = NewsData.objects.first().id
    specific_news = NewsData.objects.get(id=first_id)
    print(f"\nNews by ID {first_id}: {specific_news.title}")
except:
    print("Couldn't retrieve news by ID")


In [ ]:

# 3. Filter records
# Get news from a specific category
tech_news = NewsData.objects.filter(category="科技")
print(f"\nTechnology news count: {tech_news.count()}")
if tech_news.exists():
    print(f"Example tech news: {tech_news.first().title}")


In [ ]:

# 4. Complex filtering
# Get news from the last 7 days
one_week_ago = datetime.now().date() - timedelta(days=7)
recent_news = NewsData.objects.filter(timestamp__gte=one_week_ago)
print(f"\nNews from the last 7 days: {recent_news.count()}")


In [ ]:

# 5. Ordering
latest_news = NewsData.objects.order_by('-timestamp')[:5]  # Get 5 most recent news
print("\n5 most recent news:")
for news in latest_news:
    print(f"- {news.id} {news.timestamp}:{news.category} {news.title}")


In [ ]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title')[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")


In [ ]:
titles_only

In [ ]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title', flat=True)[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")


In [ ]:

# 7. Text search (simple contains)
keyword = "台灣"
matching_news = NewsData.objects.filter(content__contains=keyword).count()
print(f"\nNews mentioning '{keyword}': {matching_news}")

In [ ]:
from django.db.models import Q, Max, F
from collections import Counter

## 3. Uptimestamp (U) - Modifying Data

The 'Uptimestamp' operation modifies existing records in the database. There are several ways to uptimestamp data in Django.

In [ ]:
# Let's demonstrate uptimestamps with a temporary test record
test_news = NewsData.objects.create(
    title="Test News for Uptimestamp Demo",
    link="https://example.com/test-uptimestamp",
    category="測試",
    content="This is a temporary record to show uptimestamp operations.",
    timestamp=datetime.now().date()
)


In [ ]:

# Method 1: Instance uptimestamp
print(f"Original title: {test_news.title}")
test_news.title = "Uptimestampd Test News Title"
test_news.category = "更新測試"
test_news.save()
print(f"Uptimestampd title: {test_news.title}, Category: {test_news.category}")


In [ ]:

# Method 2: QuerySet uptimestamp (more efficient for multiple records)
# This uptimestamps all matching records without loading them into memory
uptimestamp_count = NewsData.objects.filter(id=test_news.id).uptimestamp(
    content="This content was uptimestampd using QuerySet.uptimestamp() method.",
    photo="https://example.com/uptimestampd-image.jpg"
)
print(f"\nRecords uptimestampd via QuerySet: {uptimestamp_count}")


In [ ]:
test_news.delete()

## 4. Delete (D) - Removing Data

The 'Delete' operation removes records from the database. Django provides various methods to delete data.

In [ ]:
# Let's demonstrate uptimestamps with a temporary test record
test_news = NewsData.objects.create(
    id="item-123",  # This will be auto-generated by the database
    title="Test News for Uptimestamp Demo",
    link="https://example.com/test-uptimestamp",
    category="刪除測試",
    content="This is a temporary record to show uptimestamp operations.",
    timestamp=datetime.now().date()
)


In [ ]:
NewsData.objects.get(id="item-123").delete()

In [ ]:
NewsData.objects.filter( category="刪除測試")

In [ ]:
NewsData.objects.filter( category="刪除測試").delete()

# filter_database_fullText

In [ ]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest timestamp in the database
    latest_timestamp = NewsData.objects.aggregate(max_timestamp=Max('timestamp'))['max_timestamp']
    
    # Calculate start timestamp
    start_timestamp = latest_timestamp - timedelta(weeks=weeks)
    
    # Base query - filter by timestamp range
    queryset = NewsData.objects.filter(timestamp__gte=start_timestamp, timestamp__lte=latest_timestamp)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        # q_objects會長這樣： Q(content__contains=kw1) | Q(content__contains=kw2) | Q(content__contains=kw3)
        # 這樣就可以用在filter裡面了
        queryset = queryset.filter(q_objects)
    
    return queryset

In [ ]:

user_keywords = ['烏克蘭', '俄羅斯']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [ ]:
for news in queryset:
    print(news.title)

In [ ]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo')[:3]

In [ ]:
news_items

In [ ]:
list(queryset.values_list('timestamp', flat=True)) # flat=True to get a flat list 否則會是 tuple

In [ ]:
latest_timestamp = NewsData.objects.aggregate(max_timestamp=Max('timestamp'))['max_timestamp']

In [ ]:
latest_timestamp

In [ ]:
from django.db.models import Avg, Count, Min, Max, Sum
from django.db.models.functions import Extract

# Aggregate是用來計算總和、平均值、最小值、最大值等統計數據的函數
# 這些函數可以用來對查詢集進行聚合計算，並返回一個字典，其中包含計算結果的鍵值對
# Basic aggregation examples
aggregations = NewsData.objects.aggregate(
    count=Count('id'),
    oldest=Min('timestamp'),
    newest=Max('timestamp'),
)
print("Basic aggregations:")
for key, value in aggregations.items():
    print(f"- {key}: {value}")


In [ ]:

# Category-based aggregation
category_counts = NewsData.objects.values('category').annotate(
    count=Count('id')
).order_by('-count')[:5]  # Top 5 categories
print("\nTop 5 categories by count:")
for item in category_counts:
    print(f"- {item['category']}: {item['count']} articles")

In [ ]:
# annotate是用來在查詢集中添加計算字段的函數
# 這些計算字段可以用來進行進一步的過濾、排序或分組操作
# values是用來選擇查詢集中的特定字段的函數 字段就是資料庫中的欄位
# timestamp-based aggregation (articles per day)
timestamp_counts = NewsData.objects.values('timestamp').annotate(
    count=Count('id')
).order_by('-timestamp')[:7]  # Last 7 days with data

print("\nArticles per day (most recent 7 days with data):")
for item in timestamp_counts:
    print(f"- {item['timestamp']}: {item['count']} articles")
